# Sessione 5 – Orchestratore Multi-Agente

Dimostra una semplice pipeline a due agenti (Ricercatore -> Editor) utilizzando Foundry Local.


### Spiegazione: Installazione delle dipendenze
Installa `foundry-local-sdk` e `openai` necessari per l'accesso ai modelli locali e per completamenti di chat. Idempotente.


# Scenario
Implementa un modello orchestratore minimale a due agenti:
- **Agente Ricercatore** raccoglie punti concisi e fattuali
- **Agente Editor** riscrive per una chiarezza esecutiva

Dimostra memoria condivisa per agente, passaggio sequenziale dell'output intermedio e una funzione pipeline semplice. Estendibile a più ruoli (es. Critico, Verificatore) o rami paralleli.

**Variabili d'Ambiente:**
- `FOUNDRY_LOCAL_ALIAS` - Modello predefinito da utilizzare (default: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Modello principale dell'agente (sovrascrive ALIAS)
- `AGENT_MODEL_EDITOR` - Modello dell'agente Editor (default: modello principale)

**Riferimento SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Come funziona:**
1. **FoundryLocalManager** avvia automaticamente il servizio Foundry Local
2. Scarica e carica il modello specificato (o utilizza la versione in cache)
3. Fornisce un endpoint compatibile con OpenAI per l'interazione
4. Ogni agente può utilizzare un modello diverso per compiti specializzati
5. La logica di retry integrata gestisce i fallimenti transitori in modo efficace

**Caratteristiche principali:**
- ✅ Scoperta e inizializzazione automatica del servizio
- ✅ Gestione del ciclo di vita del modello (download, cache, caricamento)
- ✅ Compatibilità con SDK OpenAI per un'API familiare
- ✅ Supporto multi-modello per la specializzazione degli agenti
- ✅ Gestione robusta degli errori con logica di retry
- ✅ Inferenza locale (nessuna API cloud richiesta)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Spiegazione: Importazioni Core e Tipizzazione
Introduce le dataclass per la memorizzazione dei messaggi degli agenti e suggerimenti di tipizzazione per maggiore chiarezza. Importa il gestore locale di Foundry + il client OpenAI per le azioni successive dell'agente.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Spiegazione: Inizializzazione del Modello (Pattern SDK)
Utilizza Foundry Local Python SDK per una gestione robusta dei modelli:
- **FoundryLocalManager(alias)** - Avvia automaticamente il servizio e carica il modello tramite alias
- **get_model_info(alias)** - Risolve l'alias in un ID modello concreto
- **manager.endpoint** - Fornisce l'endpoint del servizio per il client OpenAI
- **manager.api_key** - Fornisce la chiave API (opzionale per utilizzo locale)
- Supporta modelli separati per agenti diversi (principale vs editor)
- Logica di retry integrata con backoff esponenziale per maggiore resilienza
- Verifica della connessione per garantire che il servizio sia pronto

**Pattern SDK Chiave:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Gestione del Ciclo di Vita:**
- I manager sono memorizzati globalmente per una corretta pulizia
- Ogni agente può utilizzare un modello diverso per la specializzazione
- Scoperta automatica del servizio e gestione della connessione
- Retry graduale con backoff esponenziale in caso di errori

Questo garantisce una corretta inizializzazione prima che inizi l'orchestrazione degli agenti.

**Riferimento:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Spiegazione: Classi Agent & Memory
Definisce `AgentMsg` leggero per le voci di memoria e `Agent` che incapsula:
- **Ruolo del sistema** - Persona e istruzioni dell'agente
- **Cronologia dei messaggi** - Mantiene il contesto della conversazione
- **Metodo act()** - Esegue azioni con gestione corretta degli errori

L'agente può utilizzare modelli diversi (primario vs editor) e mantiene un contesto isolato per ogni agente. Questo schema consente:
- Persistenza della memoria tra le azioni
- Assegnazione flessibile dei modelli per ogni agente
- Isolamento e recupero dagli errori
- Facile concatenazione e orchestrazione


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Spiegazione: Pipeline Orchestrata
Crea due agenti specializzati:
- **Ricercatore**: Utilizza il modello primario, raccoglie informazioni fattuali
- **Editor**: Può utilizzare un modello separato (se configurato), perfeziona e riscrive

La funzione `pipeline`:
1. Il Ricercatore raccoglie informazioni grezze
2. L'Editor le perfeziona trasformandole in un output pronto per la presentazione
3. Restituisce sia i risultati intermedi che quelli finali

Questo schema consente:
- Specializzazione dei modelli (modelli diversi per ruoli diversi)
- Miglioramento della qualità attraverso un'elaborazione a più fasi
- Tracciabilità della trasformazione delle informazioni
- Facile estensione a più agenti o elaborazione parallela


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Spiegazione: Esecuzione della Pipeline & Risultati
Esegue la pipeline multi-agente su una domanda a tema conformità + latenza per dimostrare:
- Trasformazione delle informazioni in più fasi
- Specializzazione e collaborazione tra agenti
- Miglioramento della qualità dell'output attraverso la raffinazione
- Tracciabilità (sia degli output intermedi che finali conservati)

**Struttura dei Risultati:**
- `question` - Query originale dell'utente
- `research` - Output grezzo della ricerca (punti fattuali)
- `final` - Sintesi esecutiva raffinata
- `models` - Modelli utilizzati per ogni fase

**Idee di Estensione:**
1. Aggiungere un agente Critic per la revisione della qualità
2. Implementare agenti di ricerca paralleli per aspetti diversi
3. Aggiungere un agente Verifier per la verifica dei fatti
4. Utilizzare modelli diversi per livelli di complessità differenti
5. Implementare cicli di feedback per miglioramenti iterativi


### Avanzato: Configurazione Personalizzata dell'Agente

Prova a personalizzare il comportamento dell'agente modificando le variabili d'ambiente prima di eseguire la cella di inizializzazione:

**Modelli Disponibili:**
- Usa `foundry model ls` nel terminale per vedere tutti i modelli disponibili
- Esempi: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, ecc.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
